In [1]:
# --- AoC 2025 - Day 8: Playground ---

# 0. Configuration and Imports
# -----------------------------------------------------------------------------
import os
import sys
import re
from collections import defaultdict, Counter, deque
import math
import itertools

# Define input filename - typically both parts use the same 'input.txt'
INPUT_FILENAME = "input.txt"

# Set up paths for convenience
NOTEBOOK_DIR = os.getcwd() 


# 1. Load Input Data
# -----------------------------------------------------------------------------
def load_input_data(filename):
    """
    Loads input data from a specified file.
    Assumes the input file is in the same directory as the notebook.
    """
    filepath = os.path.join(NOTEBOOK_DIR, filename)
    try:
        with open(filepath, 'r') as f:
            # Read all lines and strip whitespace from each
            return [line.strip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Error: Input file '{filename}' not found at '{filepath}'")
        return [] # Return an empty list to prevent further errors

# Load the raw data once for both parts
raw_data = load_input_data(INPUT_FILENAME)

if raw_data:
    print(f"Loaded {len(raw_data)} lines from '{INPUT_FILENAME}'.")
    print(f"First 5 lines: {raw_data[:5]}\n")
else:
    print("No data loaded (or file not found). Operations will rely on test data.\n")


# =============================================================================
# >>> START PART 1 (Solve this first!) <<<
# =============================================================================

# 2. Part 1 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part1(data_lines):
    """
    Parses the raw input data for Part 1.
    Input: "162,817,812" -> Tuple (162, 817, 812)
    Returns a list of coordinate tuples.
    """
    print("Parsing data for Part 1...")
    if not data_lines: 
        return []

    points = []
    for line in data_lines:
        if not line: continue
        try:
            parts = list(map(int, line.split(',')))
            if len(parts) == 3:
                points.append(tuple(parts))
        except ValueError:
            print(f"Warning: Could not parse line: {line}")
            
    return points

parsed_input_part1 = parse_data_part1(raw_data)


# 3. Part 1 Solution Algorithm
# -----------------------------------------------------------------------------
class UnionFind:
    """Helper class for tracking connected components."""
    def __init__(self, size):
        self.parent = list(range(size))
        # We track size of each component directly
        self.size = [1] * size

    def find(self, i):
        if self.parent[i] == i:
            return i
        self.parent[i] = self.find(self.parent[i]) # Path compression
        return self.parent[i]

    def union(self, i, j):
        root_i = self.find(i)
        root_j = self.find(j)
        
        if root_i != root_j:
            # Merge smaller into larger
            if self.size[root_i] < self.size[root_j]:
                root_i, root_j = root_j, root_i
            
            self.parent[root_j] = root_i
            self.size[root_i] += self.size[root_j]
            return True
        return False

def solve_part1(data, connection_limit=1000):
    """
    Solves the first part of the puzzle.
    1. Calculate all pairwise distances.
    2. Sort by distance.
    3. 'Connect' the top N pairs.
    4. Find size of largest 3 circuits.
    """
    print("Solving Part 1...")
    
    if not data:
        return 0
        
    n = len(data)
    edges = []
    
    # 1. Calculate all pairwise distances
    # itertools.combinations(range(n), 2) gives all unique pairs of indices
    for i, j in itertools.combinations(range(n), 2):
        p1 = data[i]
        p2 = data[j]
        # Euclidean distance
        dist = math.sqrt((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)
        edges.append((dist, i, j))
        
    # 2. Sort by distance (ascending)
    edges.sort(key=lambda x: x[0])
    
    # 3. Connect top pairs
    uf = UnionFind(n)
    
    # Take only the requested number of connections
    # Note: If input has fewer pairs than limit, we take all of them.
    limit = min(len(edges), connection_limit)
    
    for k in range(limit):
        _, u, v = edges[k]
        uf.union(u, v)
        
    # 4. Analyze Circuits
    # Get all component sizes. 
    # Since we used path compression, we iterate roots or just check parents.
    # The 'size' array in our UF is only guaranteed valid for roots.
    
    component_sizes = []
    for i in range(n):
        if uf.parent[i] == i: # Is a root
            component_sizes.append(uf.size[i])
            
    # Sort sizes descending
    component_sizes.sort(reverse=True)
    
    # Multiply top 3
    # Be safe if fewer than 3 components exist (though puzzle implies many)
    result = 1
    count = 0
    for s in component_sizes:
        result *= s
        count += 1
        if count == 3:
            break
            
    return result

# Note: The real solution uses limit=1000 per the puzzle description.
# The variable is passed into the function to allow the test case (limit=10) to work.
part1_answer = solve_part1(parsed_input_part1, connection_limit=1000)
print(f"Part 1 Answer: {part1_answer}\n")


# 4. Part 1 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART1_STR = """
162,817,812
57,618,57
906,360,560
592,479,940
352,342,300
466,668,158
542,29,236
431,825,988
739,650,466
52,470,668
216,146,977
819,987,18
117,168,530
805,96,715
346,949,466
970,615,88
941,993,340
862,61,35
984,92,344
425,690,689
"""
EXAMPLE_EXPECTED_PART1 = 40

def test_part1():
    print("Running Part 1 example test...")
    if not EXAMPLE_INPUT_PART1_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART1_STR.strip().split('\n')
    example_parsed_data = parse_data_part1(example_raw_data)
    
    # IMPORTANT: The example uses the "ten shortest connections" (limit=10)
    result = solve_part1(example_parsed_data, connection_limit=10)
    
    assert result == EXAMPLE_EXPECTED_PART1, \
        f"Part 1 Example Failed! Expected {EXAMPLE_EXPECTED_PART1}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART1})")
    print("Part 1 example test finished. Circuits are buzzing!\n")

# Run the test
test_part1()


# =============================================================================
# >>> START PART 2 (Only after solving Part 1) <<<
# =============================================================================

# 5. Part 2 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part2(data_lines):
    """
    Parses the raw input data for Part 2.
    """
    print("Parsing data for Part 2...")
    if not data_lines:
        return []

    # Usually reuses Part 1 parsing
    return parse_data_part1(data_lines)

parsed_input_part2 = parse_data_part2(raw_data)


# 6. Part 2 Solution Algorithm
# -----------------------------------------------------------------------------
def solve_part2(data):
    """
    Solves the second part of the puzzle.
    """
    print("Solving Part 2...")
    
    # --- Part 2 Algorithm Here ---

    return "Part 2 Solution Not Implemented Yet"

part2_answer = solve_part2(parsed_input_part2)
print(f"Part 2 Answer: {part2_answer}\n")


# 7. Part 2 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART2_STR = EXAMPLE_INPUT_PART1_STR
EXAMPLE_EXPECTED_PART2 = 0

def test_part2():
    print("Running Part 2 example test...")
    if not EXAMPLE_INPUT_PART2_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART2_STR.strip().split('\n')
    example_parsed_data = parse_data_part2(example_raw_data)
    result = solve_part2(example_parsed_data)
    
    # assert result == EXAMPLE_EXPECTED_PART2, \
    #     f"Part 2 Example Failed! Expected {EXAMPLE_EXPECTED_PART2}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART2})")
    print("Part 2 example test finished.\n")

# Run the Part 2 test
# test_part2()

print("\n✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas")

Loaded 1000 lines from 'input.txt'.
First 5 lines: ['89590,84247,92413', '31429,76854,36723', '78677,64030,40809', '27833,56570,85542', '45295,90593,4593']

Parsing data for Part 1...
Solving Part 1...
Part 1 Answer: 330786

Running Part 1 example test...
Parsing data for Part 1...
Solving Part 1...
Test Result: 40 (Expected: 40)
Part 1 example test finished. Circuits are buzzing!

Parsing data for Part 2...
Parsing data for Part 1...
Solving Part 2...
Part 2 Answer: Part 2 Solution Not Implemented Yet


✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas
